In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import keras.backend as K
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *

from ipywidgets import *
from bqplot import LinearScale
import bqplot.pyplot as plt

from utils import r_square

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../data/bls_prices.csv')
X = data.drop('price', axis=1).values
y = data['price'].values

# split data into train and val sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=2000)

In [ ]:
# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
class TrainingPlotsDashboard(VBox):
    """
    dashboard for training plots (loss/accuracy curves)
    """
    def __init__(self, *args, **kwargs):
        self.epochs = kwargs['epochs']
        self.width = kwargs.get('width', 960)
        self.height = kwargs.get('height', 500)
        self.layout = Layout(width=str(self.width) + 'px',
                             height=str(self.height) + 'px')

        kwargs['children'] = self.build_widgets()
        super(TrainingPlotsDashboard, self).__init__(*args, **kwargs)

    def build_widgets(self):
        # loss curve
        self.loss_fig = plt.figure(title='Loss Curve')
        axes_options = {'y': {'label': 'Loss', 
                              'tick_format': '.1f',
                              'label_offset': '-1em',
                              'label_location': 'end'},
                        'x': {'label': 'Epochs'}}
        self.loss_plot = plt.plot([], [], colors=['orangered', 'limegreen'],
                                  axes_options=axes_options,
                                  display_legend=True,
                                  labels=['Train', 'Test'])

        # accuracy curve
        self.accuracy_fig = plt.figure(title='Accuracy Curve')
        plt.scales(scales={'y': LinearScale(min=.99, max=1)})
        axes_options = {'y': {'label': 'R Square',
                              'tick_format': '.2%',
                              'label_offset': '-1em',
                              'label_location': 'end'},
                        'x': {'label': 'Epochs'}}
        self.accuracy_plot = plt.plot([], [],
                                      colors=['orangered', 'limegreen'],
                                      axes_options=axes_options,
                                      display_legend=True,
                                      labels=['Train', 'Test'])
        
        self.progress_bar = IntProgress(description='Training Progress', 
                                        min=0, max=(self.epochs - 1),
                                        layout=Layout(margin='10px 0px 10px 60px'))
        return [self.progress_bar, HBox([self.loss_fig, self.accuracy_fig])]

    def clear_plots(self):
        self.loss_plot.x = []
        self.accuracy_plot.x = []

In [ ]:
class TrainingCallback(Callback):
    def __init__(self, *args, **kwargs):
        self.dashboard = kwargs['dashboard']

    def on_train_begin(self, epoch, logs={}):
        self.epochs = []
        self.train_loss = []
        self.test_loss = []

        self.train_acc = []
        self.test_acc = []

        self.epoch_weights = []
        self.epoch_biases = []
        self.epoch_activations = []
        self.epoch_gradient_norms = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.dashboard.progress_bar.value = epoch
        self.epochs.append(epoch + 1)
        
        self.train_loss.append(logs['loss'])
        
        if 'acc' in logs:
            self.train_acc.append(logs['acc'])
        elif 'r_square' in logs:
            self.train_acc.append(logs['r_square'])            
            
        if 'val_loss' in logs:
            self.test_loss.append(logs['val_loss'])

        if 'val_acc' in logs:
            self.test_acc.append(logs['val_acc'])
        elif 'val_r_square' in logs:
            self.test_acc.append(logs['val_r_square'])
        
        # update dashboard plots
        # loss plot
        with self.dashboard.loss_plot.hold_sync():
            self.dashboard.loss_plot.x = self.epochs
            if len(self.test_loss) > 0:
                self.dashboard.loss_plot.y = [self.train_loss, self.test_loss]
            else:
                self.dashboard.loss_plot.y = self.train_loss

        # accuracy plot
        with self.dashboard.accuracy_plot.hold_sync():
            self.dashboard.accuracy_plot.x = self.epochs

            if len(self.test_acc) > 0:
                self.dashboard.accuracy_plot.y = [self.train_acc,
                                                  self.test_acc]
            else:
                self.dashboard.accuracy_plot.y = self.train_acc

In [ ]:
epochs = 100
dashboard = TrainingPlotsDashboard(epochs=epochs, width=1200, height=550)
training_callback = TrainingCallback(dashboard=dashboard)
dashboard

In [ ]:
activation = 'relu'

model = Sequential()
model.add(Dense(100, input_dim=4, activation=activation))
model.add(Dense(50, activation=activation))
model.add(Dense(10, activation=activation))
model.add(Dense(1))
optimizer = Adam()
model.compile(optimizer=optimizer, loss='mse', metrics=[r_square, 'mae'])

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=epochs, batch_size=64, verbose=0,
                    callbacks=[training_callback])